# Get Tweets

This script extracts all the tweets with hashtag #covid-19 related to the day before today (yesterday) and saves them into a .csv file.
We use the `tweepy` library, which can be installed with the command `pip install tweepy`.

Firstly, we import the configuration file, called `config.py`, which is located in the same directory of this script.

In [23]:
!pip3 install tweepy

You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
from config import *
import tweepy
import datetime

We setup the connection to our Twitter App by using the `OAuthHandler()` class and its `access_token()` function. Then we call the Twitter API through the `API()` function.

In [26]:
auth = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True)

Now we setup dates. We need to setup today and yesterday.

In [27]:
today = datetime.date.today()
yesterday= today - datetime.timedelta(days=1)
today, yesterday

(datetime.date(2021, 3, 22), datetime.date(2021, 3, 21))

We search for tweets on Twitter by using the `Cursor()` function. 
We pass the `api.search` parameter to the cursor, as well as the query string, which is specified through the `q` parameter of the cursor.
The query string can receive many parameters, such as the following (not mandatory) ones:
* `from:` - to specify a specific Twitter user profile
* `since:` - to specify the beginning date of search
* `until:` - to specify the ending date of search
The cursor can also receive other parameters, such as the language and the `tweet_mode`. If `tweet_mode='extended'`, all the text of the tweet is returned, otherwise only the first 140 characters.

In [28]:
tweets_list = tweepy.Cursor(api.search, q="#Covid-19 since:" + str(yesterday)+ " until:" + str(today),tweet_mode='extended', lang='it').items()

Now we loop across the `tweets_list`, and, for each tweet, we extract the text, the creation date, the number of retweets and the favourite count. We store every tweet into a list, called `output`.

In [29]:
output = []
for tweet in tweets_list:
    text = tweet._json["full_text"]
    print(text)
    favourite_count = tweet.favorite_count
    retweet_count = tweet.retweet_count
    created_at = tweet.created_at
    
    line = {'text' : text, 'favourite_count' : favourite_count, 'retweet_count' : retweet_count, 'created_at' : created_at}
    output.append(line)

RT @Cirenderaliberi: PROSEGUONO I "SUICIDI DI STATO" 

https://t.co/VbMBZTUyoJ https://t.co/bkFv4alfOF
@Consu84 Sta lavorando.. il progetto esiste e lei è ambasciatrice. 

https://t.co/HPs6bFRUcd
@GingerPresident Il progetto esiste veramente, non si è inventata proprio nulla
https://t.co/HPs6bFRUcd
RT @Vic5211: Guenda non si è inventata proprio nulla.. Almeno se l'avessero fatta parlare
https://t.co/HPs6bFRUcd 
 #noneladurso
Guenda non si è inventata proprio nulla.. Almeno se l'avessero fatta parlare
https://t.co/HPs6bFRUcd 
 #noneladurso
RT @MinervaMcGrani1: Allora amici se mi dimentico qualcosa aggiungete pure alla lista

Con il #Covid_19 sono riusciti a : fare fuori Trump,…
RT @RobertoBurioni: Vi aspetto questa sera, per spiegarvi come si stabilisce che un vaccino è sicuro ed efficace, in modo che possiate vacc…
RT @donTonio66: Ai "furbetti del vaccino" e a chi ritiene che il #Covid_19 sia cosa da niente: domani, alle 16.30, venite al cimitero del m…
RT @borghi_claudio: @Corss80 @Fe

RT @borghi_claudio: @Corss80 @Ferdi__Romano @PagellaPolitica Amico, non sono le contee, qui si sta discutendo dei provvedimenti annunciati…
RT @borghi_claudio: @Corss80 @Ferdi__Romano @PagellaPolitica Amico, non sono le contee, qui si sta discutendo dei provvedimenti annunciati…
RT @borghi_claudio: @Corss80 @Ferdi__Romano @PagellaPolitica Comunque se proprio lo chiede ecco qui il brillante piano del North Dakota, co…
@Corss80 @Ferdi__Romano @PagellaPolitica Comunque se proprio lo chiede ecco qui il brillante piano del North Dakota, con il codice a colori e le regolette per ogni colore. Ricorda qualcosa?
https://t.co/vxdsH3pyEz
Niente di tutto ciò è stato fatto in South Dakota.
Differenze? Nessuna.
RT @borghi_claudio: @Corss80 @Ferdi__Romano @PagellaPolitica Amico, non sono le contee, qui si sta discutendo dei provvedimenti annunciati…
RT @borghi_claudio: @Corss80 @Ferdi__Romano @PagellaPolitica Amico, non sono le contee, qui si sta discutendo dei provvedimenti annunciati…
RT @_cieloita

KeyboardInterrupt: 

In [30]:
output

[{'text': 'RT @Cirenderaliberi: PROSEGUONO I "SUICIDI DI STATO" \n\nhttps://t.co/VbMBZTUyoJ https://t.co/bkFv4alfOF',
  'favourite_count': 0,
  'retweet_count': 10,
  'created_at': datetime.datetime(2021, 3, 21, 23, 51, 21)},
 {'text': '@Consu84 Sta lavorando.. il progetto esiste e lei è ambasciatrice. \n\nhttps://t.co/HPs6bFRUcd',
  'favourite_count': 0,
  'retweet_count': 0,
  'created_at': datetime.datetime(2021, 3, 21, 23, 48, 31)},
 {'text': '@GingerPresident Il progetto esiste veramente, non si è inventata proprio nulla\nhttps://t.co/HPs6bFRUcd',
  'favourite_count': 0,
  'retweet_count': 0,
  'created_at': datetime.datetime(2021, 3, 21, 23, 45, 2)},
 {'text': "RT @Vic5211: Guenda non si è inventata proprio nulla.. Almeno se l'avessero fatta parlare\nhttps://t.co/HPs6bFRUcd \n #noneladurso",
  'favourite_count': 0,
  'retweet_count': 3,
  'created_at': datetime.datetime(2021, 3, 21, 23, 45)},
 {'text': "Guenda non si è inventata proprio nulla.. Almeno se l'avessero fatta parlare\

Finally, we convert the `output` list to a `pandas DataFrame` and we store results.

In [36]:
output

[{'text': 'RT @Cirenderaliberi: PROSEGUONO I "SUICIDI DI STATO" \n\nhttps://t.co/VbMBZTUyoJ https://t.co/bkFv4alfOF',
  'favourite_count': 0,
  'retweet_count': 10,
  'created_at': datetime.datetime(2021, 3, 21, 23, 51, 21)},
 {'text': '@Consu84 Sta lavorando.. il progetto esiste e lei è ambasciatrice. \n\nhttps://t.co/HPs6bFRUcd',
  'favourite_count': 0,
  'retweet_count': 0,
  'created_at': datetime.datetime(2021, 3, 21, 23, 48, 31)},
 {'text': '@GingerPresident Il progetto esiste veramente, non si è inventata proprio nulla\nhttps://t.co/HPs6bFRUcd',
  'favourite_count': 0,
  'retweet_count': 0,
  'created_at': datetime.datetime(2021, 3, 21, 23, 45, 2)},
 {'text': "RT @Vic5211: Guenda non si è inventata proprio nulla.. Almeno se l'avessero fatta parlare\nhttps://t.co/HPs6bFRUcd \n #noneladurso",
  'favourite_count': 0,
  'retweet_count': 3,
  'created_at': datetime.datetime(2021, 3, 21, 23, 45)},
 {'text': "Guenda non si è inventata proprio nulla.. Almeno se l'avessero fatta parlare\

In [42]:
import pandas as pd

df = pd.DataFrame(output)
df.to_csv('output.csv', mode='a', header=False)
#df.to_csv('output.csv')

In [38]:
df.shape

(75, 4)